In [59]:
import numpy as np
import pandas as pd
from clean import manifest_clinical_merge

In [60]:
pd.set_option('display.max_columns', 100)

In [61]:
manifest_df = pd.read_csv('../Manifest/GCD_TARGET_Data_Manifest_AML_NBL_WT_RT.csv')

wt_disc_df = pd.read_excel('../Clinical_Data/TARGET_WT_ClinicalData_Discovery_20160714_public.xlsx')
aml_disc_df = pd.read_excel('../Clinical_Data/TARGET_AML_ClinicalData_20160714.xlsx')
nbl_disc_df = pd.read_excel('../Clinical_Data/TARGET_NBL_ClinicalData_20151124.xlsx')

WT_df = manifest_clinical_merge(manifest_df, wt_disc_df, 'TARGET-WT')
AML_df = manifest_clinical_merge(manifest_df, aml_disc_df, 'TARGET-AML')
NBL_df = manifest_clinical_merge(manifest_df, nbl_disc_df, 'TARGET-NBL')

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/clean.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  target_df['TARGET USI'] = target_df.loc[:, 'entity_submitter_id'].apply(lambda x: x[:16])


In [62]:
from clean import assay_transpose

In [256]:
assay_df = pd.read_csv('../Expn_Data/TARGET_NBL_AML_RT_WT_TMMCPM_log2_Norm_Counts.csv.zip')
assay_t_df = assay_transpose(assay_df)

In [64]:
from clean import assay_clinical_merge

In [65]:
AML_genes = assay_clinical_merge(assay_t_df, AML_df)
WT_genes = assay_clinical_merge(assay_t_df, WT_df)
NBL_genes = assay_clinical_merge(assay_t_df, NBL_df)

In [66]:
# AML_genes.head()

# Using AML_genes

In [239]:
from model_comp import data_prep_columns, model_prep, model_comp, model_prep_loc, xgboost_tuner

In [68]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

### picking genes by highest variance between low risk / not low risk

In [251]:
df, a = data_prep_columns(AML_genes, 'Max')

In [253]:
df['Risk group'].value_counts()

Standard    69
Low         60
High         8
Name: Risk group, dtype: int64

In [72]:
X_train, X_test, y_train, y_test, holdout = model_prep(df, a)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()


In [73]:
model_comp(df, X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.30893145582567405
Gradient Boost Log Loss 0.36210740308746076
Random Forest Log Loss 0.3262005795045274
XGBoost Log Loss 0.26104056291483546


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


### repeating with tuned xgboost

In [250]:
df, a = data_prep_columns(AML_genes, 'Max')
X_train, X_test, y_train, y_test, holdout = model_prep(df, a)
model_comp(df, X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.28771180389607903
Gradient Boost Log Loss 0.36210740308746076
Random Forest Log Loss 0.3262005795045274
XGBoost Log Loss 0.22184269636164647


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
        max_depth=3, min_child_weight=1, missing=None, n_estimators=700,
        n_jobs=1, nthread=None, objective='binary:logistic', random_state=8,
        reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
        silent=True, subsample=1),
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=3, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
             oob_score=False, random_state=8, verbose=1, warm_start=False))

### taking highest variance is log loss of .26

In [74]:
df, a = data_prep_columns(AML_genes, 'Min')

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137 entries, 3 to 185
Columns: 21492 entries, TARGET USI to Low Risk
dtypes: float64(3), int64(7), object(21482)
memory usage: 22.5+ MB


In [77]:
X_train, X_test, y_train, y_test, holdout = model_prep(df, a)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()


In [79]:
model_comp(df, X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.4428831437875592
Gradient Boost Log Loss 0.49256451159202913
Random Forest Log Loss 0.48223471620602953
XGBoost Log Loss 0.38563264594287483


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


### model performs much worse than taking genes with the lowest variance of the two groups

### comparing against random gene selection

In [31]:
df.iloc[:, 84:-4].head()

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,ENSG00000001460.16,ENSG00000001461.15,ENSG00000001497.15,ENSG00000001561.6,ENSG00000001617.10,ENSG00000001626.13,ENSG00000001629.8,ENSG00000001630.14,ENSG00000001631.13,ENSG00000002016.15,ENSG00000002330.12,ENSG00000002549.11,ENSG00000002586.16,ENSG00000002587.8,ENSG00000002726.18,ENSG00000002746.13,ENSG00000002822.14,ENSG00000002834.16,ENSG00000002919.13,ENSG00000002933.6,ENSG00000003056.6,ENSG00000003096.12,ENSG00000003137.7,ENSG00000003147.16,ENSG00000003249.12,ENSG00000003393.13,ENSG00000003400.13,ENSG00000003402.18,ENSG00000003436.13,ENSG00000003509.14,ENSG00000003756.15,ENSG00000003987.12,ENSG00000003989.15,ENSG00000004059.9,ENSG00000004139.12,ENSG00000004142.10,ENSG00000004399.11,ENSG00000004455.15,ENSG00000004468.11,ENSG00000004478.7,...,ENSG00000280789.1,ENSG00000280798.1,ENSG00000280828.1,ENSG00000280832.1,ENSG00000280913.1,ENSG00000280916.1,ENSG00000280927.1,ENSG00000280936.1,ENSG00000280953.1,ENSG00000280987.1,ENSG00000281005.1,ENSG00000281021.1,ENSG00000281026.1,ENSG00000281100.1,ENSG00000281103.1,ENSG00000281106.1,ENSG00000281128.1,ENSG00000281131.1,ENSG00000281183.1,ENSG00000281189.1,ENSG00000281195.1,ENSG00000281205.1,ENSG00000281207.1,ENSG00000281332.1,ENSG00000281344.1,ENSG00000281357.1,ENSG00000281358.1,ENSG00000281376.1,ENSG00000281383.1,ENSG00000281392.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281406.1,ENSG00000281433.1,ENSG00000281445.1,ENSG00000281468.1,ENSG00000281490.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281538.1,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1
3,-1.78095,-6.12136,3.64096,3.709,2.38266,6.39166,2.39788,4.65639,3.84851,4.54683,1.43006,3.98078,4.68736,3.60569,1.96781,-6.92715,4.49665,-0.459083,4.34997,2.9013,3.31713,4.96146,7.05126,-2.01182,0.639075,-6.92715,3.10366,6.75977,3.14635,0.968573,5.71694,1.80983,-4.92989,3.91116,-2.08518,4.79765,4.44668,7.07993,3.3462,2.92602,6.29794,2.08949,0.384373,5.23568,2.18425,4.98321,6.21358,7.13245,6.91584,4.7842,...,0.87496,2.77872,2.02122,1.07344,-2.42284,0.314777,1.74517,-6.92714,-6.12136,1.40683,0.995511,-0.223406,2.2914,-0.613341,3.9011,1.61224,-4.6821,-6.92714,0.8846,-2.4712,0.732776,0.678398,0.391151,1.82242,1.4786,1.37983,0.894175,-1.11056,-0.330849,0.0915962,1.95643,-0.547052,-6.92714,-1.45498,1.12314,-2.93057,2.9388,1.76878,0.580973,-0.832714,-1.78095,0.132623,-0.682824,5.46561,-0.330849,-1.84323,-0.801464,-0.364707,2.1646,-0.152689
5,-2.0565,-4.74701,4.01127,1.7086,2.45773,8.33617,-0.390612,5.86531,2.81586,4.3824,2.7385,2.93014,5.38245,0.610766,-0.00724986,-6.92715,2.7025,0.592369,2.33861,2.51256,4.33478,5.21782,8.32612,-2.53384,1.66084,-1.48807,4.79562,6.58635,4.58207,3.27681,6.57761,0.929309,-5.18099,1.85467,-1.56858,1.28478,3.38364,5.60581,-0.251195,1.83133,5.2888,-0.740952,-2.38062,6.86317,2.74684,6.33174,7.06771,7.84417,5.86555,6.0705,...,2.52717,1.46242,1.00775,3.60854,-5.80459,-6.92714,1.03526,-6.92714,-6.92714,0.113774,1.32433,-4.41384,0.664587,-6.92714,1.21435,-6.92714,-6.92714,-5.80459,-0.0939682,-2.53384,-1.48807,-4.14335,-1.05214,0.929309,0.340124,0.95113,1.78753,-0.202203,6.32096,-0.544956,1.04881,0.496696,-6.92714,-2.79931,-0.914236,-1.94482,1.12118,3.16766,-2.61702,-1.89207,-3.1249,1.05554,-1.94482,3.83862,-2.17756,-0.717812,-0.88817,1.75482,0.456577,0.965496
9,-0.600677,-6.92715,4.33729,3.70407,1.45783,5.80541,3.08307,4.70263,5.93286,3.68418,1.49483,3.81156,4.45815,5.90521,2.60909,-6.92715,3.98982,0.267381,3.63427,1.96923,3.15996,5.24463,8.50432,-1.56366,-0.0547583,-5.86665,2.9938,7.11946,4.06171,-0.879721,5.81253,0.494469,-4.50961,3.90215,-0.239935,4.13606,4.74356,6.5404,5.79849,3.05547,6.71508,2.10406,-3.35954,5.6307,2.3536,5.15138,6.21046,5.87504,5.85272,2.3282,...,0.370532,2.74585,2.34096,-1.029

In [110]:
random_cols = np.random.choice(21404, 2000)

In [111]:
len(np.unique(random_cols))

1918

In [112]:
random_set = set(random_cols)
random_cols = list(random_set)

In [113]:
len(random_cols)

1918

In [114]:
random_cols_ = [x + 84 for x in random_cols]

In [115]:
# df.iloc[random_cols]

In [116]:
X_train, X_test, y_train, y_test, holdout = model_prep(df, random_cols_)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()


In [117]:
model_comp(df, X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.44553880177733396
Gradient Boost Log Loss 0.4461835161204546
Random Forest Log Loss 0.4368195141425212
XGBoost Log Loss 0.5289622393210192


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


# Lasso 

In [137]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression

In [172]:
lasso_columns = df.iloc[:, 84:-4].columns

In [178]:
X_train, X_test, y_train, y_test, holdout = model_prep_lasso(df, lasso_columns)

In [193]:
# log_model = Pipeline([('standardize', StandardScaler()),
#                     ('classifier', LogisticRegression(penalty='l1', tol=0.01, solver='saga'))])

In [194]:
log_model = LogisticRegression(penalty='l1', solver='saga')

In [195]:
# X_train

In [196]:
log_model.fit(X_train, y_train)

/Users/david/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [197]:
y_pred = log_model.predict_proba(X_test)
log_loss(y_test, y_pred)

0.2126451489896612

In [200]:
log_coefs = np.array(log_model.coef_)

In [201]:
len(log_coef)

21404

In [203]:
mask = log_coefs > 0

In [211]:
important_genes = lasso_columns[mask[0]]

In [212]:
important_genes

Index(['ENSG00000002586.16', 'ENSG00000002726.18', 'ENSG00000002933.6',
       'ENSG00000003137.7', 'ENSG00000003249.12', 'ENSG00000003436.13',
       'ENSG00000005102.11', 'ENSG00000005108.14', 'ENSG00000005243.8',
       'ENSG00000005381.7',
       ...
       'ENSG00000280777.1', 'ENSG00000280953.1', 'ENSG00000281106.1',
       'ENSG00000281404.1', 'ENSG00000281433.1', 'ENSG00000281468.1',
       'ENSG00000281501.1', 'ENSG00000281527.1', 'ENSG00000281571.1',
       'ENSG00000281741.1'],
      dtype='object', length=3128)

# Using those columns with trees

In [240]:
X_train, X_test, y_train, y_test, holdout = model_prep_loc(df, important_genes)

In [223]:
xg, rf = model_comp(df, X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.4004047191019699
Gradient Boost Log Loss 0.4342166231893474
Random Forest Log Loss 0.39916905372456085
XGBoost Log Loss 0.43392028045412656


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [231]:
xg_important = important_genes[np.array(xg.feature_importances_) > 0]

In [232]:
rf_important = important_genes[np.array(rf.feature_importances_) > 0]

In [233]:
print(len(rf_important))
len(xg_important)

927


75

In [236]:
X_train, X_test, y_train, y_test, holdout = model_prep_lasso(df, rf_important)
xg1, rf1 = model_comp(df, X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3647851773898809
Gradient Boost Log Loss 0.37493114140076117
Random Forest Log Loss 0.3773670012218379
XGBoost Log Loss 0.3765728272296287


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [237]:
X_train, X_test, y_train, y_test, holdout = model_prep_lasso(df, xg_important)
xg2, rf2 = model_comp(df, X_train, X_test, y_train, y_test)

Ensemble Log Loss 0.36974907632539183
Gradient Boost Log Loss 0.37572774176747037
Random Forest Log Loss 0.36150705103422376
XGBoost Log Loss 0.43392028045412656


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


# XG tuning

In [246]:
X_train, X_test, y_train, y_test, holdout = model_prep_loc(df, important_genes)

In [247]:
xgboost_tuner(X_train, X_test, y_train, y_test, [6, 7, 8, 9, 10])

XGB n_estimators: 600 log loss 0.3840244703900975
XGB n_estimators: 700 log loss 0.3819335944497505
XGB n_estimators: 800 log loss 0.38423496384072947
XGB n_estimators: 900 log loss 0.38454268512794293
XGB n_estimators: 1000 log loss 0.3847021654348921


In [248]:
# best number of trees is 700